<a href="https://colab.research.google.com/github/geersenthil/Topic-Modeling-/blob/main/NNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import LsiModel
from gensim.parsing import strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords
from gensim.parsing import preprocess_string
from nltk.tokenize import RegexpTokenizer

from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

#nltk
import nltk 
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords


#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet


# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#Spacy
import spacy


#Dataset
from sklearn.datasets import fetch_20newsgroups


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
news_group = fetch_20newsgroups()

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [3]:
news_df = pd.DataFrame({'news': news_group_data})
news_data =  pd.DataFrame({'news':news_df['news'].sample(500)})
print(news_data.head())

                                                    news
7846   From: mouse@thunder.mcrcim.mcgill.edu (der Mou...
1143   From: Feng.Qian@launchpad.unc.edu (Feng Qian)\...
2898   From: franjion@spot.Colorado.EDU (John Franjio...
10036  From: s851708@minyos.xx.rmit.OZ.AU (John Edmon...
959    From: cjhs@minster.york.ac.uk\nSubject: Re: Xt...


Pre-process the data Lemmatization to get base words

In [4]:
def lemmatization(texts, allowed_postags=["NOUN","ADJ","VERB","ADV"]):
  nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return (texts_out)

In [5]:
lemmatizated_text =  lemmatization(news_data['news'])
print(lemmatizated_text)

['subject always opaque parent child relation line distribution comp misuse newsgroup hierarchy name distribution article write wonder possible parent window paint area child attribute window attribute use drawing Set subwindow mode rather default', 'subject owner miss deal tape post host lambada.oit.unc.edu suggest use pre - format tape tape drive often can successfully format tape need bulk erase can format again new preformatte tape tape drive accuTRAK MB ea never use ea wrap email interested opinion express necessarily internet laUNChpad.unc.edu', 'franjion@spot subject area medium leafs coverage posting host spot.colorado.edu line write write area media report tonight beat Can part medium conspiracy tell how game really go expect win shutout hat trick goal assist really lose how many penalty whichever biased game call let win area medium report rumour lead candidate replace fire coach apparently only covet win attitude play coach will expect quarterback powerplay good comeback qui

In [6]:
stop_words = stopwords.words('english')
print(len(stop_words))
stop_words.extend(['from', 'subject', 're', 'edu', 'use','cdw','would','line','article'])
print(len(stop_words))

179
188


In [9]:
def clean_text(headline):
  word_tokens=word_tokenize(headline)
  tokens=[w for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [11]:
news_data['news_cleaned_text']=news_data['news'].apply(clean_text)

In [12]:
news_data.head()

,news,news_cleaned_text
7846,From: mouse@thunder.mcrcim.mcgill.edu (der Mou...,From mouse thunder.mcrcim.mcgill.edu Mouse Sub...
1143,From: Feng.Qian@launchpad.unc.edu (Feng Qian)\...,From Feng.Qian launchpad.unc.edu Feng Qian Sub...
2898,From: franjion@spot.Colorado.EDU (John Franjio...,From franjion spot.Colorado.EDU John Franjione...
10036,From: s851708@minyos.xx.rmit.OZ.AU (John Edmon...,From s851708 minyos.xx.rmit.OZ.AU John Edmond ...
959,From: cjhs@minster.york.ac.uk\nSubject: Re: Xt...,From cjhs minster.york.ac.uk Subject intrinsic...
